In [4]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch import optim

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
df.shape

(569, 33)

In [7]:
df.drop(columns=["id", "Unnamed: 32"], inplace=True)

In [8]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=.2, random_state=42)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [12]:
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [13]:
X_test_tensor

tensor([[-0.4665, -0.1373, -0.4442,  ..., -0.1944,  0.1728,  0.2037],
        [ 1.3654,  0.4987,  1.3055,  ...,  0.9918, -0.5612, -1.0084],
        [ 0.3801,  0.0692,  0.4041,  ...,  0.5704, -0.1078, -0.2063],
        ...,
        [-0.7355, -0.9985, -0.7414,  ..., -0.2774, -0.3821, -0.3241],
        [ 0.0290,  2.0334,  0.0275,  ..., -0.4903, -1.6091, -0.3314],
        [ 1.8722,  2.8008,  1.8035,  ...,  0.7926, -0.0587, -0.0947]])

In [28]:
from torch.utils.data import DataLoader, Dataset

class custd(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.label = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.label[index]

In [29]:
train_dataset = custd(X_train_tensor, y_train_tensor)
test_dataset = custd(X_test_tensor, y_test_tensor)

In [30]:
train_dataset[9]

(tensor([ 1.6343,  0.2523,  1.5895,  1.5784,  0.5721,  0.5224,  0.6576,  1.2107,
         -0.0327, -0.1217,  0.3743, -0.6079,  0.3846,  0.4160, -0.6223, -0.1402,
         -0.0717, -0.0181, -1.2255, -0.1543,  1.5595, -0.0505,  1.6017,  1.3562,
          0.4410,  0.5048,  0.6880,  0.7221, -0.5644,  0.4202]),
 tensor(1.))

In [31]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [16]:
class myNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)

        return out



In [17]:
lr = 0.1
epochs = 250

loss_fun = nn.BCELoss()


In [32]:
model = myNN(X_train_tensor.shape[1])


optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(epochs):
    for batch_fea, batch_lab in train_loader:    
    
        y_pred = model(batch_fea)
        
        loss = loss_fun(y_pred, batch_lab.view(-1,1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    

    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.21723468601703644
Epoch 2, Loss: 0.2324211150407791
Epoch 3, Loss: 0.21120478212833405
Epoch 4, Loss: 0.32370254397392273
Epoch 5, Loss: 0.10292287915945053
Epoch 6, Loss: 0.23351362347602844
Epoch 7, Loss: 0.06462014466524124
Epoch 8, Loss: 0.02531387098133564
Epoch 9, Loss: 0.03219398111104965
Epoch 10, Loss: 0.037611670792102814
Epoch 11, Loss: 0.027275873348116875
Epoch 12, Loss: 0.48075947165489197
Epoch 13, Loss: 0.05612766370177269
Epoch 14, Loss: 0.017354099079966545
Epoch 15, Loss: 0.04233647137880325
Epoch 16, Loss: 0.059351615607738495
Epoch 17, Loss: 0.008781379088759422
Epoch 18, Loss: 0.041249845176935196
Epoch 19, Loss: 0.03069167397916317
Epoch 20, Loss: 0.055864326655864716
Epoch 21, Loss: 0.010211394168436527
Epoch 22, Loss: 0.10432548820972443
Epoch 23, Loss: 0.002902251435443759
Epoch 24, Loss: 0.1271994560956955
Epoch 25, Loss: 0.050702761858701706
Epoch 26, Loss: 0.10380135476589203
Epoch 27, Loss: 0.07484166324138641
Epoch 28, Loss: 0.14173313975

In [36]:
model.eval()
acc_list = []


with torch.no_grad():
    for batch_fea, batch_lab in test_loader:
        y_pred = model.forward(batch_fea)
        y_pred = (y_pred > 0.5).float()
        batch_acc = (y_pred == batch_lab).float().mean()
        acc_list.append(batch_acc)

total_acc = sum(acc_list) / len(acc_list)

    

In [37]:
total_acc

tensor(0.5395)